In [11]:
import plotly.express as px
import plotly.graph_objects as go
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import requests
import pandas as pd
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
from datetime import datetime
import urllib.request

import ssl
import json
import time

>Dokumnetacja-> https://lunarcrush.com/developers/docs#


>login -> fozurek@wp.pl


> hasło -> StockApp

>widgets - > https://lunarcrush.com/developers/widgets

In [12]:
coins = ['BTC','DOT','ETH','DOGE','LTC','ADA','XRP']

In [13]:
def get_coin_data(symbol,data_points=365,interval='day'):
    url = "https://api.lunarcrush.com/v2?data=assets&key=39o5gbbtx7n1gxkvnhf6na&symbol={}&data_points={}&interval={}".format(symbol,data_points,interval)
    response = requests.get(url).json()
    data = response['data'][0]
    df = pd.json_normalize(data['timeSeries'])
    df['time'] = pd.to_datetime(df['time'],unit='s')
    data = pd.json_normalize(data)
    details = data[['name','symbol','price_btc','market_cap','percent_change_24h','percent_change_7d','percent_change_30d','volume_24h','max_supply']]

    return (df,details.to_dict(orient='records')[0])
  

# Test space 

In [14]:
x,x_details = get_coin_data('DOT',data_points=365,interval='day')  

In [15]:
# x_details = x_details.to_dict(orient='records')[0]
# x_details['name']

In [26]:
fig = go.Figure(data=[go.Candlestick(x=x['time'],
                open=x['open'],
                high=x['high'],
                low=x['low'],
                close=x['close'])])
fig.update_layout(title=x_details['name'],
                 yaxis_tickformat = '$',
                 xaxis_title="Date",
                 yaxis_title="Price",)
fig.show()

In [24]:
df = px.data.tips()
# Build App
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("StockAPP"),
    dcc.Graph(id='graph'),
    html.Label([
        "Coin",
        dcc.Dropdown(
            id='coin-dropdown', clearable=False,
            value='coin', options=[
                {'label': c, 'value': c}
                for c in coins
            ])
    ]),
])
# Define callback to update graph
@app.callback(
    Output('graph', 'figure'),
    [Input("coin-dropdown", "value")]
)
def update_figure(coin):
    x,details = get_coin_data(coin)
#     x_details.to_dict(orient='records')[0]
    fig = go.Figure(data=[go.Candlestick(x=x['time'],
                open=x['open'],
                high=x['high'],
                low=x['low'],
                close=x['close'])])
    fig.update_layout(title=x_details['name'],
                 yaxis_tickformat = '$',
                 xaxis_title="Date",
                 yaxis_title="Price",)
    return fig
#     return px.scatter(
#         df, x="total_bill", y="tip", color="size",
#         color_continuous_scale=colorscale,
#         render_mode="webgl", title="Tips"
#     )
# Run app and display result inline in the notebook


#by odpalisc dasha na stornie zmien 'inlline' na 'external '
app.run_server(mode='inline')